KPI 1: Net Rating (Rating Neto)
Este código calculará las posesiones, el Offensive Rating (ORtg) y el Defensive Rating (DRtg) para cada partido.

In [ ]:
import pandas as pd
import numpy as np

# Cargar el archivo de datos de partidos limpios
# Asegúrate de que la ruta del archivo sea correcta
df_games = pd.read_csv(r"C:\Users\sebas\OneDrive\Escritorio\Henry\processed_data\clean_game.csv")

# --- 1. CÁLCULO DE POSESIONES (ESTIMACIÓN) ---

# Fórmula de posesiones (aproximada, la más común en analítica NBA):
# POSS = FGA - OREB + TOV + 0.44 * FTA
# Aquí se calcula la posesión para el equipo de Local (Home) y el visitante (Away)
# Y se toma el promedio para el partido.

# Para el equipo de Local (Home)
# Usamos oreb_home, tov_home, fga_home, fta_home del archivo clean_game.csv
df_games['POSS_HOME'] = (
    df_games['fga_home'] 
    - df_games['oreb_home'] 
    + df_games['tov_home'] 
    + 0.44 * df_games['fta_home']
)

# Para el equipo VISITANTE (Away)
# Usamos oreb_away, tov_away, fga_away, fta_away del archivo clean_game.csv
df_games['POSS_AWAY'] = (
    df_games['fga_away'] 
    - df_games['oreb_away'] 
    + df_games['tov_away'] 
    + 0.44 * df_games['fta_away']
)

# La posesión total del juego es el promedio, ya que los equipos comparten la posesión
df_games['POSS'] = (df_games['POSS_HOME'] + df_games['POSS_AWAY']) / 2

# --- 2. CÁLCULO DE ORtg, DRtg y Net Rating ---

# 2.1. Cálculos para el equipo de CASA
# ORtg Home = (Puntos Anotados / Posesiones) * 100
df_games['ORtg_Home'] = (df_games['pts_home'] / df_games['POSS']) * 100
# DRtg Home = ORtg del oponente (Puntos Permitidos / Posesiones) * 100
df_games['DRtg_Home'] = (df_games['pts_away'] / df_games['POSS']) * 100
# Net Rating Home
df_games['NetRtg_Home'] = df_games['ORtg_Home'] - df_games['DRtg_Home']


# 2.2. Cálculos para el equipo VISITANTE
# ORtg Away = (Puntos Anotados / Posesiones) * 100
df_games['ORtg_Away'] = (df_games['pts_away'] / df_games['POSS']) * 100
# DRtg Away = ORtg del oponente (Puntos Permitidos / Posesiones) * 100
df_games['DRtg_Away'] = (df_games['pts_home'] / df_games['POSS']) * 100
# Net Rating Away
df_games['NetRtg_Away'] = df_games['ORtg_Away'] - df_games['DRtg_Away']


# --- 3. AGREGACIÓN DE LOS NUEVOS KPIS POR EQUIPO Y TEMPORADA ---

# Paso 3a: Preparar los datos para la agregación por equipo (Stacking)

# Seleccionamos las columnas relevantes y renombramos para apilarlas
home_cols = [
    'season_id', 'team_id_home', 'NetRtg_Home', 
    'ORtg_Home', 'DRtg_Home'
]
away_cols = [
    'season_id', 'team_id_away', 'NetRtg_Away', 
    'ORtg_Away', 'DRtg_Away'
]

# Creamos DataFrames separados y renombramos las columnas para estandarizar
df_home = df_games[home_cols].rename(columns={
    'team_id_home': 'team_id',
    'NetRtg_Home': 'NetRtg',
    'ORtg_Home': 'ORtg',
    'DRtg_Home': 'DRtg'
})

df_away = df_games[away_cols].rename(columns={
    'team_id_away': 'team_id',
    'NetRtg_Away': 'NetRtg',
    'ORtg_Away': 'ORtg',
    'DRtg_Away': 'DRtg'
})

# Combinamos los datos de local y visitante
df_aggregated = pd.concat([df_home, df_away])

# Paso 3b: Calcular los promedios de Net Rating por equipo y temporada
df_kpis_net_rating = df_aggregated.groupby(['season_id', 'team_id']).agg(
    Avg_NetRtg=('NetRtg', 'mean'),
    Avg_ORtg=('ORtg', 'mean'),
    Avg_DRtg=('DRtg', 'mean'),
    Total_Games=('team_id', 'size')
).reset_index()

# Mostrar las primeras filas del nuevo KPI
print("--- NUEVO KPI: Net Rating Promedio por Equipo y Temporada ---")
print(df_kpis_net_rating.head(10))

# Exportar para usar en BigQuery/Looker (opcional)
# df_kpis_net_rating.to_csv('nba_kpi_net_rating.csv', index=False) -El archivo ya fue creado y se cambio de carpeta

--- NUEVO KPI: Net Rating Promedio por Equipo y Temporada ---
   season_id     team_id  Avg_NetRtg    Avg_ORtg    Avg_DRtg  Total_Games
0      12005  1610612737   -0.011710  105.235222  105.246932            7
1      12005  1610612738   -3.647793   97.264026  100.911819            8
2      12005  1610612739    5.098488  100.960431   95.861943            8
3      12005  1610612740   -2.735808  101.656246  104.392054            7
4      12005  1610612741    0.217446  100.062212   99.844766            7
5      12005  1610612742    0.124842   98.561244   98.436402            8
6      12005  1610612743    6.512998  102.831881   96.318883            8
7      12005  1610612744   -3.445342  104.765161  108.210503            7
8      12005  1610612745    6.440203   96.051124   89.610921            7
9      12005  1610612746   10.678749  104.021556   93.342808            7


KPI 2: True Shooting Percentage (TS%)El TS% mide la eficiencia de tiro de un jugador o equipo, contando los tiros libres y de tres puntos.$$\text{TS\%} = \frac{\text{Puntos Anotados} \times 50}{\text{FGA} + (0.44 \times \text{FTA})}$$Este KPI es más útil en el archivo de equipos/jugadores agregados, pero se puede calcular a nivel de juego y luego promediar.

In [ ]:
# Continuamos usando df_games del código anterior

# --- CÁLCULO DE TRUE SHOOTING PERCENTAGE (TS%) ---

# Para el equipo de CASA (Home)
# pts_home, fga_home, fta_home
df_games['TS_PCT_Home'] = (
    df_games['pts_home'] * 50
) / (
    df_games['fga_home'] + (0.44 * df_games['fta_home'])
)

# Para el equipo VISITANTE (Away)
# pts_away, fga_away, fta_away
df_games['TS_PCT_Away'] = (
    df_games['pts_away'] * 50
) / (
    df_games['fga_away'] + (0.44 * df_games['fta_away'])
)

# --- AGREGACIÓN DE TS% POR EQUIPO Y TEMPORADA ---

# Paso 1: Preparar los datos para la agregación por equipo (Stacking)

# Seleccionamos las columnas relevantes y renombramos para apilarlas
home_ts_cols = ['season_id', 'team_id_home', 'TS_PCT_Home']
away_ts_cols = ['season_id', 'team_id_away', 'TS_PCT_Away']

df_ts_home = df_games[home_ts_cols].rename(columns={
    'team_id_home': 'team_id',
    'TS_PCT_Home': 'TS_PCT'
})

df_ts_away = df_games[away_ts_cols].rename(columns={
    'team_id_away': 'team_id',
    'TS_PCT_Away': 'TS_PCT'
})

# Combinamos los datos de local y visitante
df_ts_aggregated = pd.concat([df_ts_home, df_ts_away])

# Paso 2: Calcular el promedio de TS% por equipo y temporada
df_kpis_ts_pct = df_ts_aggregated.groupby(['season_id', 'team_id']).agg(
    Avg_TS_PCT=('TS_PCT', 'mean')
).reset_index()

# Mostrar las primeras filas del nuevo KPI
print("\n--- NUEVO KPI: True Shooting % Promedio por Equipo y Temporada ---")
print(df_kpis_ts_pct.head(10))

# Exportar para usar en BigQuery/Looker (opcional)
# df_kpis_ts_pct.to_csv('nba_kpi_ts_pct.csv', index=False) -El archivo ya fue creado y se cambio de carpeta


--- NUEVO KPI: True Shooting % Promedio por Equipo y Temporada ---
   season_id     team_id  Avg_TS_PCT
0      12005  1610612737   57.455647
1      12005  1610612738   55.162898
2      12005  1610612739   54.290982
3      12005  1610612740   53.896251
4      12005  1610612741   50.837698
5      12005  1610612742   53.624849
6      12005  1610612743   56.586141
7      12005  1610612744   50.922210
8      12005  1610612745   53.954063
9      12005  1610612746   54.011137


Influencia del Árbitro (referee_effect)
El objetivo es calcular si un equipo, históricamente, gana más o menos partidos que su promedio con un árbitro específico.

In [ ]:
import pandas as pd

# 1. EXTRACCIÓN Y CARGA DE DATOS (Extract & Load)
# Asegúrate de que las rutas de tus archivos sean correctas
df_game = pd.read_csv(r"C:\Users\sebas\OneDrive\Escritorio\Henry\processed_data\clean_game.csv")
df_officials = pd.read_csv(r"C:\Users\sebas\OneDrive\Escritorio\Henry\processed_data\officials_clean.csv")
df_teams = pd.read_csv(r"C:\Users\sebas\OneDrive\Escritorio\Henry\processed_data\team.csv")[['id', 'full_name']] 

# --- 2. TRANSFORMACIÓN (Feature Engineering: Referee Effect) ---

# 2.1. UNIR PARTIDOS Y ÁRBITROS
# Unimos el resultado del partido (clean_game) con el árbitro que lo pitó (officials_clean)
df_game_ref = pd.merge(
    df_game[['game_id', 'wl_home', 'team_id_home', 'team_id_away', 'season_id']], 
    df_officials[['game_id', 'official_id']], 
    on='game_id', 
    how='inner'
)
# Convertir 'W'/'L' a numérico (1/0)
df_game_ref['HOME_WINS'] = df_game_ref['wl_home'].apply(lambda x: 1 if x == 'W' else 0)

# 2.2. Estandarizar la tabla por FILA DE EQUIPO
# Duplicamos las filas para tener una entrada por EQUIPO en lugar de por PARTIDO

# Datos para el equipo LOCAL
df_home = df_game_ref.rename(columns={
    'team_id_home': 'team_id',
    'team_id_away': 'opponent_id'
})
df_home['is_home'] = 1
df_home['is_win'] = df_home['HOME_WINS']

# Datos para el equipo VISITANTE
df_away = df_game_ref.rename(columns={
    'team_id_away': 'team_id',
    'team_id_home': 'opponent_id'
})
df_away['is_home'] = 0
df_away['is_win'] = df_away['HOME_WINS'].apply(lambda x: 1 - x) # Si el local perdió (0), el visitante ganó (1)

# Concatenar para tener una tabla de EQUIPO-PARTIDO-ÁRBITRO
df_team_game_ref = pd.concat([df_home, df_away], ignore_index=True)

# 2.3. CALCULAR TASAS DE VICTORIA GENERALES (Benchmark)

# Calcular la Tasa de Victoria (Win Rate) general de cada equipo
df_win_rate_general = df_team_game_ref.groupby('team_id').agg(
    General_Win_Rate=('is_win', 'mean')
).reset_index()

# 2.4. CALCULAR TASA DE VICTORIA POR ÁRBITRO
# Calcular el Win Rate del equipo cuando el árbitro específico está presente
df_win_rate_referee = df_team_game_ref.groupby(['team_id', 'official_id']).agg(
    Team_Games_with_Ref=('game_id', 'count'),
    Team_Wins_with_Ref=('is_win', 'sum')
).reset_index()

df_win_rate_referee['Ref_Win_Rate'] = (
    df_win_rate_referee['Team_Wins_with_Ref'] / df_win_rate_referee['Team_Games_with_Ref']
)

# 2.5. CÁLCULO FINAL DEL 'REFEREE EFFECT' (Unir y Restar)

# Unir el Win Rate general con el Win Rate por árbitro
df_final_effect = pd.merge(
    df_win_rate_referee, 
    df_win_rate_general, 
    on='team_id', 
    how='left'
)

# El Feature: Referee Effect = (Win Rate con Árbitro) - (Win Rate General)
df_final_effect['referee_effect'] = (
    df_final_effect['Ref_Win_Rate'] - df_final_effect['General_Win_Rate']
)

# --- 3. EXPORTACIÓN (Load) ---

# Filtrar para mostrar solo los datos esenciales
df_output = df_final_effect[['team_id', 'official_id', 'referee_effect']].sort_values(
    'referee_effect', ascending=False
)

# Unir con nombres de equipos y árbitros para mejor visualización
df_output = pd.merge(df_output, df_teams, left_on='team_id', right_on='id', how='left').rename(columns={'full_name': 'Team_Name'})

print("--- TOP 5 INFLUENCIAS POSITIVAS DE ÁRBITROS POR EQUIPO ---")
# Muestra que árbitros favorecen más a cada equipo (Referee Effect positivo alto)
# Este es el dato que alimenta el Feature 'referee_effect' en la matriz
print(df_output.head())

# Guardar el nuevo feature para usarlo en BigQuery/Looker
# df_output.to_csv('feature_referee_effect.csv', index=False) -El archivo ya fue creado y se cambio de carpeta

--- TOP 5 INFLUENCIAS POSITIVAS DE ÁRBITROS POR EQUIPO ---
   team_id  official_id  referee_effect  id Team_Name
0    12308         1174        0.739130 NaN       NaN
1    12308       202007        0.739130 NaN       NaN
2    12308         1188        0.739130 NaN       NaN
3    12308         2001        0.739130 NaN       NaN
4    12315         1174        0.705882 NaN       NaN
